In [1]:
# baseline cnn model for digit
import numpy as np
#import import_ipynb
from loader import Loader
from sklearn.model_selection import KFold
from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import BatchNormalization
import random as r
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array, load_img
from matplotlib import pyplot
import scipy.stats as stats
import os
from keras.utils.vis_utils import plot_model


In [2]:
def dataset1():
# Dividing the data into train and test data
    #clear = lambda: os.system('clear')
    #clear()
    
    dataset = Loader()
    noise = 1
    print(noise)
    
    trainX,trainY =  dataset.getWholeTrainSet(pca=False) 
    
    
    trainXaug,trainYaug =  dataset.augment() 
    
    trainXaug1 = np.concatenate((trainX, trainXaug))
    trainYaug1 = np.concatenate((trainY, trainY))
    print(trainYaug1.shape)
    trainXwithnoise, trainYwithnoise = dataset.getNoisySet(intensity=noise, set="train", flat=False)
    
    testX, testY = dataset.getWholeTestSet()
    
    trainX = np.array(trainX)
    trainXwithnoise = np.array(trainXwithnoise)
    trainY = np.array(trainY)
    testX = np.array(testX)
    testY = np.array(testY)
   
    trainXwithnoise = trainXwithnoise.reshape((trainXwithnoise.shape[0], 16,15, 1))
    trainXaug1 =  trainXaug1.reshape((trainXaug1.shape[0], 16,15, 1))
    testX = testX.reshape((testX.shape[0], 16,15, 1))
    trainX = trainX.reshape((trainX.shape[0], 16,15, 1))
    # the labels are one hot encoded
    testYintegers = testY
    trainYaug1 =  to_categorical(trainYaug1)
    testY = to_categorical(testY)
    trainY = to_categorical(trainY)
    return trainXaug1,trainXwithnoise, trainYaug1, testX, testY,testYintegers

In [3]:
def getdataset1(intensity,dataset):
    
    trainX,trainXwithnoise, trainY, testX, testY,testYintegers = dataset1(intensity,dataset)
    return trainX,trainXwithnoise, trainY, testX, testY,testYintegers

In [4]:
def model_definition():
    # tune model parameters here
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(16,15, 1)))
    #model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    
    
    
    # Flatten and apply drop out or apply drop out after Conv2D it is essentially the same
    model.add(Dropout(0.50))
    model.add(Dense(240, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    
    model.add(Dropout(0.20))
    model.add(Dense(120, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    model.add(Dropout(0.20))
    model.add(Dense(50, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    
    
    
    
#    model.add(Dropout(0.20))
    model.add(Dense(10, activation='softmax'))
    #opt = SGD(lr=0.01)
    opt = SGD(lr=0.03, momentum=0.9)
    #opt = 'adam'
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
model = model_definition()

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 13, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 13, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 6, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1344)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1344)              0         
_________________________________________________________________
dense (Dense)                (None, 240)               322800    
_________________________________________________________________
batch_normalization_1 (Batch (None, 240)               9

In [7]:
def train_modelkfold(mtrainX,mtrainXwithnoise, mtrainY, folds=10):
    accuracies,histories = list(), list()

    kfold = KFold(folds, shuffle=True,random_state=3)      
    print(mtrainX.shape)
    print(mtrainY.shape)
    for training_indices, testing_indices in kfold.split(mtrainX):
        model = model_definition()
        trainX, trainY, testX, testY = mtrainX[training_indices], mtrainY[training_indices], mtrainX[testing_indices], mtrainY[testing_indices]
        # so this model.fit reflects the model.fit for only the kfold 
        #model.fit(trainX, trainY, epochs=120, batch_size=10, validation_data=(testX, testY), verbose=0)
        history_kfold = model.fit(trainX, trainY, epochs=200, batch_size=32, validation_data=(testX, testY), verbose=0)
        
        # returning the validation accuracies of every k fold validation set
        _, accuraccy = model.evaluate(testX, testY, verbose=0)
        print('%.3f' % (accuraccy * 100.0))
        # stores accuracies and test data on last kth model
        accuracies.append(accuraccy)
        histories.append(history_kfold)
        # implement a list for storing all the k fold models. If you wish
    return accuracies,histories

In [8]:
# function to print the accuracies of the k fold iterations
def fitmodel(trainX, trainY,testX, testY,i):   
    
    #once the model has been decided on from k fold
    model = model_definition()
    model.fit(trainX, trainY,epochs=200, verbose=0)
    # save model
    name = "final_model.h5" + str(i)
    model.save(name)
    # load the model
    model = load_model(name)
    return model

In [9]:
def modelpredictions(modellist,testX,testY):
    
    
    correctpernoise = []
    for model in modellist:
        correctaveragepermodel = 0.0
        correct = 0.0
        preds = model.predict(testX)
        print(preds.shape)
        preds = np.argmax(model.predict(testX), axis=-1)
        #preds = to_categorical(preds)
        #print(preds)
        
        for i in range(len(testY)):
            if (preds[i] == testY[i]): # Change
                correct = correct+1
                
        correctaveragepermodel = correct/len(testY)
        correctpernoise.append(correctaveragepermodel)
        print(correctaveragepermodel)
        #return list
    return correctpernoise

In [10]:
def results():
    modellist = []
    finalresultmodelnonoiselist = []
    numofmodels = 10
    #intensities = [.05, 0.4]#, .1, .15, .20, .25, .30, .35, .40, .45, .50]
    
    #datasetlist = [Loader() for i in range(2)]
    modellist = []
    trainX,trainXwithnoise, trainY, testX, testY,testYintegers = dataset1()
    print(trainY.shape)
    # trainXwithnoise1,train_norm1, test_norm1= normalise_data(trainXwithnoise,trainX,testX) 
    print('printing the k fold accuracies with noisy training data')
    accuracies, histories = train_modelkfold(trainX,trainXwithnoise, trainY)  
    #accuracies, histories = train_modelkfold(trainX,trainXwithnoise, trainY)

    finalresultpernoise = 0
    # making 5 models
    for i in range(numofmodels):
        model = fitmodel(trainX, trainY,testX, testY,i)
        modellist.append(model)

    correct = 0    
    correctpermodel = modelpredictions(modellist,testX,testYintegers)


    finalresultmodelnonoise = (sum(correctpermodel)/numofmodels)*100
    finalresultmodelnonoiselist.append(finalresultmodelnonoise)
    ###########################################################################
    #trainXwithnoise = trainXwithnoise.reshape((trainXwithnoise.shape[0], 16,15, 1))
    #testX = testX.reshape((testX.shape[0], 16,15, 1))
    #trainX = trainX.reshape((trainX.shape[0], 16,15, 1))
    #train_norm = train_norm.reshape((trainX.shape[0], 16,15, 1))
    ###########################################################################
    %reset -f in
    return finalresultmodelnonoiselist
    #predslist.append(preds)

In [11]:
result = results()

1
(2000,)
(2000, 10)
printing the k fold accuracies with noisy training data
(2000, 16, 15, 1)
(2000, 10)
100.000
99.500
99.000
99.000
98.500
100.000
99.500
100.000
99.500
100.000
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: final_model.h50/assets
INFO:tensorflow:Assets written to: final_model.h51/assets
INFO:tensorflow:Assets written to: final_model.h52/assets
INFO:tensorflow:Assets written to: final_model.h53/assets
INFO:tensorflow:Assets written to: final_model.h54/assets
INFO:tensorflow:Assets written to: final_model.h55/assets
INFO:tensorflow:Assets written to: final_model.h56/assets
INFO:tensorflow:Assets written to: final_model.h57/assets
INFO:tensorflow:Assets written to: final_model.h58/assets
INFO:tensorflow:Assets written to: final_model.h59/assets
(1000,

In [12]:
result

[98.33999999999999]